# MHKiT WPTO Hindcast Example

This example loads data from the WPTO hindcast data set, hosted on AWS.  

  ## Dataset Description

The high-spatial-resolution dataset (hereafter the *'spatial dataset'*) spans the U.S. Exclusive Economic Zone (EEZ) with an unstructured grid that has ~200 m resolution in shallow water. The time step resolution for this spatial data is 3-hours and spans 32 years, 01/01/1979  - 12/31/2010.

The spatial dataset is organized into yearly files (each file contains all variables and locations in that year) and they are located on AWS at: 
        **`/nrel/US_wave/US_wave_{year}.h5`**


- Dataset variables included are indexed by **'coordinates'** (latitude and longitude), and a **'time_index'**

### Included Spatial Variables: 

| Variable Name | Units |
| :------------ | :---: |
| energy_period | seconds |
| maximum_energy_period | degrees_true |
| mean_absolute_period | seconds |
| mean_zero-crossing_period | seconds |
| omni-directional_wave_power | Watts |
| peak_period | seconds |
| significant_wave_height | meters |
| water_depth | meters |
| spectral_width | none |
| directionality_coefficient | none |


### Setting up Access to WPTO Hindcast Data
To access the WPTO hindcast data, you will need to configure h5pyd for data access on HSDS. 
        To get your own API key, visit https://developer.nrel.gov/signup/. 

To configure h5phd type:

    hsconfigure
        
and enter at the prompt:

    hs_endpoint = https://developer.nrel.gov/api/hsds
    hs_username = None
    hs_password = None
    hs_api_key = {your key}

 
## Using the WPTO Hindcast MHKiT Function
Start by importing MHKiT and the necessary python packages.

In [1]:
from mhkit import wave

## 1. Download WPTO Hindcast Data from a Single Location

You can download variables from the WPTO high-spatial-resolution dataset by specifting the file for the year(s) you are interested in, latitude/longitude pairs, and the parameter.


In [2]:
# We will start by downloading the significant wave height

file = f'/nrel/US_wave/US_wave_1995.h5' #specifying the year file of interest
lat_lon = (44.624076,-124.280097) # setting lat/lon pair of interest
parameter = 'significant_wave_height' #Requesting significant wave height

Hs, meta= wave.io.wpto_hindcast.request_wpto_dataset(file,parameter,lat_lon) #calling on WPTO hindcast MHKiT function

In [3]:
# View meta data
meta


,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
413889,77.429497,44.624298,-124.278999,15622.175781,-8,Federal


In [4]:
# View first few entries in significant wave height pd.Series 
Hs.head()

,significant_wave_height_44.624076_-124.280097
time_index,
1995-01-01 00:00:00+00:00,2.35354
1995-01-01 03:00:00+00:00,2.39468
1995-01-01 06:00:00+00:00,2.45756
1995-01-01 09:00:00+00:00,2.55913
1995-01-01 12:00:00+00:00,2.66992


## 2. Download WPTO Hindcast Data from Multiple Locations

Multiple locations can be requested by passing a list or tuple of lat/lon pairs.  

In [5]:
# Next we will download the energy period from 2 locations 
parameter = 'energy_period'
lat_lon = ((44.624076,-124.280097),(43.489171,-125.152137)) 
Te, meta= wave.io.wpto_hindcast.request_wpto_dataset(file,parameter,lat_lon) #calling on WPTO hindcast MHKiT function

# View Te from two locations
Te.head()

,energy_period_44.624076_-124.280097,energy_period_43.489171_-125.152137
time_index,,
1995-01-01 00:00:00+00:00,10.3433,10.1760
1995-01-01 03:00:00+00:00,10.2281,10.2641
1995-01-01 06:00:00+00:00,10.0164,10.1792
1995-01-01 09:00:00+00:00,10.0484,10.2410
1995-01-01 12:00:00+00:00,10.6579,10.5877


In [6]:
meta

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
413889,77.429497,44.624298,-124.278999,15622.175781,-8,Federal
296246,1337.407959,43.496300,-125.152000,64236.390625,-8,Federal


## 3. Download WPTO Hindcast Data from Multiple Years

Multiple years can be requested by passing a list of files or by using a wildcard character (*). 

In [7]:
file = f'/nrel/US_wave/US_wave_*.h5' #specifying the years file of interest
years = [1995,1996]
parameter = 'omni-directional_wave_power'
lat_lon = (44.624076,-124.280097) # setting lat/lon pair of interest
J, meta= wave.io.wpto_hindcast.request_wpto_dataset(file,parameter,lat_lon,years=years) #calling on WPTO hindcast MHKiT function

In [8]:
J

,omni-directional_wave_power_44.624076_-124.280097
time_index,
1995-01-01 00:00:00+00:00,30134.0
1995-01-01 03:00:00+00:00,30805.0
1995-01-01 06:00:00+00:00,31709.0
1995-01-01 09:00:00+00:00,34476.0
1995-01-01 12:00:00+00:00,39892.0
...,...
1996-12-31 09:00:00+00:00,154849.0
1996-12-31 12:00:00+00:00,120543.0
1996-12-31 15:00:00+00:00,108288.0


In [9]:
meta

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
413889,77.429497,44.624298,-124.278999,15622.175781,-8,Federal
